In [20]:
import numpy as np
import cv2

In [21]:
# Decimal to Binary convertor
def dec2bin(dec):
    inverse = ''
    binary = ''
    while dec != 0:
        inverse = inverse + str( int(dec % 2) )
        dec = int(dec / 2)
    if len(inverse) <= 8:
        size = len(inverse)
        for i in range(8-size):
            inverse = inverse + '0'
    for i in range(len(inverse)-1,-1,-1):
        binary = binary + str(inverse[i])
    return binary

In [22]:
# Binary to Decimal convertor
def bin2dec(binar):
    binar = int(binar)
    dec = 0
    i = 0
    while binar > 0:
        dec = dec + (int(binar%10)*np.power(2,i))
        i = i+1
        binar = int(binar/10)
    return dec

In [23]:
# XOR operation
def XOR(data,key):
    data = str(data)
    #print("data ",data)
    key = str(key)
    #print("key ",key)
    enc_data = ''
    for ext_data in range(len(data)):
        if data[ext_data] == '0' and key[ext_data] == '0':
            enc_data = enc_data + '0'
        elif data[ext_data] == '0' and key[ext_data] == '1':
            enc_data = enc_data + '1'
        elif data[ext_data] == '1' and key[ext_data] == '0':
            enc_data = enc_data + '1'
        elif data[ext_data] == '1' and key[ext_data] == '1':
            enc_data = enc_data + '0'
    return enc_data

In [24]:
def modul(d1):
    if d1 < 0:
        d1 = d1**2
        d1 = int(np.sqrt(d1))
    return d1

In [25]:
# Secret Message into ASCII and ASCII into Binary format
file = open('input.txt','r')
secret_message = ''
for line in file.readlines():
    secret_message = secret_message + str(line)
binary = ''
key = ''
key = input("Enter a numeric key of 4 digits.. ")
count = 0
tmp = ''
while True:
    if key.isnumeric() and (len(key) == 4):
        sum1 = 0
        for i in range(len(key)):
            sum1 = sum1+ ord(key[i])
        key = str(dec2bin(sum1))
        print("Converting Message to Binary and encrypting...")
        for pos in range(len(secret_message)):
            if ord(secret_message[pos]) <= 255:
                if count < 8:
                    binary = binary + str(XOR(dec2bin(ord(secret_message[pos])),key)) # coverting Secret code into ASCII and ASCII into Encrypted Binary.
                    count = count + 1
                else:
                    tmp = count = 0
        print("\nMessage Conversion Successfull.\n")
        #print("Binary Code = ",binary)
        break;
    else:
        print("please enter only 4 numeric digits the key")
        key = input("Enter a numeric key of 4 digits..")


Enter a numeric key of 4 digits..  7607


Converting Message to Binary and encrypting...

Message Conversion Successfull.



In [26]:
img = cv2.imread('cover.jpeg')
height,width,channels = img.shape
print(width,height,channels)

960 1280 3


In [28]:
bin_ct1 = bin_ct2 = 0
steagno_img = np.zeros((height,width,3),np.uint8)
for i in range(1,height):
    for j in range(0,int(width/2)*2,2):
        
        p,t1,t2 = img[i,j]
        p1,t1,t2 = img[i,j+1]
        
        #p[0],t1,t2 = img[i-1, j-1]
        #p[1],t1,t2 = img[i, j-1]
        #p[2],t1,t2 = img[i+1, j-1]
        #p[3],t1,t2 = img[i-1, j]
        #p[4],t1,t2 = img[i, j]
        #p[5],t1,t2 = img[i+1, j]
        #p[6],t1,t2 = img[i-1, j+1]
        #p[7],t1,t2 = img[i, j+1]
        #p[8],t1,t2 = img[i+1, j+1]
        
        d1 = p - p1 
        d1 = modul(d1) # Difference of the both p and p1.
        
        l1 = min(p1,p) # Lower range of the difference d1.
        u1 = max(p1,p) # Upper range of the difference d1.
        
        #print("u1 = ",u1,'\nl1 = ',l1,'\n',(u1-l1))
        if (u1-l1) != 0:
            m = int(np.log2(u1-l1)) # Number of embiding bits.
        else:
            m = 0
        
        p_psudo = 0
        p1_psudo = 0
        if m > 0 :
            bin_ct1 = bin_ct2
            bin_ct2 = bin_ct2 + m
            #print(bin_ct1,'\n',bin_ct2)
            #print(binary[bin_ct1:bin_ct2])
            secret_data = int(bin2dec(binary[bin_ct1:bin_ct2]))
            #bin_ct = bin_ct + m
            d1_psudo = l1 + secret_data
            
            if p >= p1 and d1_psudo > d1:
                p_psudo = p + int(modul(d1_psudo - d1)/2)
                p1_psudo = p1 - int(modul(d1_psudo - d1)/2)
            
            elif p < p1 and d1_psudo > d1:
                p_psudo = p - int(modul(d1_psudo - d1)/2)
                p1_psudo = p1 + int(modul(d1_psudo - d1)/2)
            
            elif p >= p1 and d1_psudo <= d1:
                p_psudo = p - int(modul(d1_psudo - d1)/2)
                p1_psudo = p1 + int(modul(d1_psudo - d1)/2)
                
            elif p < p1 and d1_psudo <= d1:
                p_psudo = p + int(modul(d1_psudo - d1)/2)
                p1_psudo = p1 - int(modul(d1_psudo - d1)/2)
            
            steagno_img[i,j] = p_psudo,p_psudo,p_psudo
            steagno_img[i,j+1] = p1_psudo,p1_psudo,p1_psudo
            
cv2.imwrite('steagno_image.jpeg',steagno_img)

C:\Users\himes\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in ubyte_scalars


ValueError: invalid literal for int() with base 10: ''

In [29]:
cv2.imshow('test',steagno_img)
cv2.waitKey(0)
cv2.destroyAllWindows()